In [10]:
%load_ext autoreload
%autoreload 2
import dataset
import folders 
import constants

import pathlib
import mne_bids
import ast
import numpy as np
import json
from collections import OrderedDict

import raw2xdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
def convert_to_BIDS(ds, data_folder: pathlib.Path) -> None:
    '''
    serious cleanup required
    '''
    def clean(a):
        if type(a) == str:
            a = a.replace('-', '').replace('_', '').replace('/', '')
        if a == "":
            a = None
        return a

    for record in ds.markup:
        record_markup = []
        epoch_counter_record = 0

        targets = ast.literal_eval(record['targets'])
        fingers = ast.literal_eval(record['fingers'])

        epochs_targets = []
        epochs_ids = []
        epochs_fingers = []
        sessions_id = []

        folder = data_folder / record['user'] / record['folder']
        files_dict = ds.get_files(folder)
        raw, chunked_events = ds.open_single_folder_eeg(files_dict,
            ignore_events_id=ast.literal_eval(record['ignore_events_id']),
            targets=targets,
            events_offset=constants.events_offset)
        events = np.vstack(chunked_events)
        raw = ds.preprocessing.process_raw_eeg(raw)

        hr_events = ds.ecg_analysis_routine(raw, folder, record, events)

        epochs = ds.preprocessing.create_epochs(raw, events)
        assert len(chunked_events) == len(targets), \
            'number of events is not equal to number of targets'
        session_id=0
        for chunk, target in zip(chunked_events, targets):
            epochs_chunk_id = 0
            for event in chunk:
                epochs_targets.append(target)
                epochs_ids.append(epochs_chunk_id)
                epochs_fingers.append(fingers[event[1]])
                sessions_id.append(session_id)
                epochs_chunk_id += 1
            session_id += 1

        assert  len(epochs) == len(events) and \
                len(epochs_targets) == len(epochs_fingers) and \
                len(epochs) == len(epochs_fingers) and \
                len(epochs_ids) == len(epochs), \
                'something is f-d up, fix it asap'

        # if hr_events is not None:
        #     plot_events = np.r_[hr_events, events]
        # else:
        #     plot_events = events
        # raw.plot(events=plot_events,
        #     event_color={constants.Rpeak_event:'green', constants.rejected_Rpeak_event:'red', -1:'#fae5ac'},
        #     block=True, scalings={'ecg':1e-4, 'misc':1e2}, start=0, duration=5)

        for epoch, event, target, finger, session_id in zip(epochs, events, epochs_targets, epochs_fingers, sessions_id):
            if hr_events is not None:
                r_deltas = hr_events[:,0] - event[0]
                ms_before_r = np.min(r_deltas[r_deltas>=0])*constants.ms_factor
                ms_after_r = abs(np.max(r_deltas[r_deltas<0])*constants.ms_factor)
            else:
                ms_after_r = None
                ms_before_r = None
                

            epoch_markup_line = OrderedDict([('onset', (event[0]-raw.first_samp) / raw.info['sfreq']),
                                             ('duration', 0),
                                             ('trial_type', event[-1]),
                                             ('value', event[-2]),
                                             ('sample', event[0]-raw.first_samp),
                                             ('target', target),
                                             ('finger', fingers[event[-2]]),
                                             ('epoch_id', epochs_ids[epoch_counter_record]),
                                             ('session_id', session_id),
                                            ])

            epoch_markup_line.update(record)

            [epoch_markup_line.pop(k, None) for k in ['ignore_events_id', 'targets', 'folder']]
            epoch_markup_line = OrderedDict([(a, clean(b)) for a, b in epoch_markup_line.items()])
              
            record_markup.append(epoch_markup_line)
        record_markup = OrderedDict( [ (key, [ev[key] for ev in record_markup]) for key in record_markup[0].keys() ] )
          
        eeg, chunked_events = ds.open_single_folder_eeg(files_dict,
                          ignore_events_id=ast.literal_eval(record['ignore_events_id']),
                          targets=targets,
                          events_offset=constants.events_offset, return_ndarray=True)

        f = mne_bids.make_bids_folders(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], kind='eeg',
              bids_root=ds.database, make_dir=True, overwrite=False, verbose=False)
          
        fn = mne_bids.make_bids_basename(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], task='bci',
              prefix=f, suffix='events.tsv')
        mne_bids.write._write_tsv(fn, record_markup, overwrite=True, verbose=False)
          
        fn = mne_bids.make_bids_basename(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], task='bci',
              prefix=f, suffix='channels.tsv')
        mne_bids.write._channels_tsv(raw, fn, overwrite=True)
          
        fn = mne_bids.make_bids_basename(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], task='bci',
              prefix=f, suffix='electrodes.tsv')
        mne_bids.write._electrodes_tsv(raw, fn, kind='eeg', overwrite=True)

        fn = mne_bids.make_bids_basename(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], task='bci',
              prefix=f, suffix='eeg.bdf')
        raw2xdf.write_npy_edf(eeg[1:,:], raw.info, fn, overwrite=True)

        eegjson = { "TaskName": "bci",
              "InstitutionName": "Lomonosov Moscow State University | ANO SensorTech",
              "InstitutionAddress": "Leninskie gory, 1, 12, Moscow, Russia",
              "Manufacturer": "Medical Computer Systems",
              "ManufacturersModelName": "NVX-52",
              "SoftwareVersions": "1.4.8639 x64",
              "CogAtlasID": "http://www.cognitiveatlas.org/task/id/tsk_GxjZBNiJorj1K",
              "CapManufacturer": "Medical Computer Systems",
              "CapManufacturersModelName": "MCScap 10-10",
              "EEGChannelCount": constants.ch_types.count('eeg'),
              "ECGChannelCount": constants.ch_types.count('ecg'),
              "EMGChannelCount": constants.ch_types.count('emg'),
              "EOGChannelCount": constants.ch_types.count('eog'),
              "MiscChannelCount": constants.ch_types.count('ecg'),
              "TriggerChannelCount": constants.ch_types.count('stim'),
              "RecordingDuration": len(raw)/raw.info['sfreq'],
              "RecordingType": "continuous",
              "EEGGround": "AFz",
              "EEGPlacementScheme": str(constants.ch_names),
              "HardwareFilters": "n/a",
              "EEGReference": "A1A2",
              "SamplingFrequency": raw.info['sfreq'],
              "PowerLineFrequency": 50,
              "SoftwareFilters": "n/a"}
        fn = mne_bids.make_bids_basename(subject=epoch_markup_line['user'], session=epoch_markup_line['reg'], task='bci',
              prefix=f, suffix='eeg.json')
        with open(fn, 'w') as f:
            json.dump(eegjson, f, indent=2)


In [ ]:
epd = dataset.EpDatasetCreator( markup_path=folders.markup_path,
                          database_path=folders.bids_path,
                          data_folder=folders.raw_data_folder,
                          reference_mode=[],
                          ICA=False,
                          fit_with_additional_lowpass=False,
                          create_database_folder=False
                          )
convert_to_BIDS(epd, data_folder = folders.raw_data_folder)